## Quando Machine Learning Encontra a Química Ambiental: Uma Comparação entre Modelo Cinético e Regressão por ML na Persistência do Mancozebe no Solo

Este projeto nasce da oportunidade de integrar conhecimentos adquiridos no mestrado em Química Ambiental com técnicas modernas de Machine Learning, avaliando até que ponto modelos orientados por dados podem complementar — ou desafiar — abordagens cinéticas clássicas na análise da persistência de pesticidas no solo.

![capa (imagem ilustrativa)](capa.png)
**Nota:** Imagem gerada por Inteligência Artificial, utilizada exclusivamente para fins ilustrativos e educacionais.

## Introdução

A avaliação da **persistência e da lixiviação de pesticidas no solo** é um tema central em química ambiental, uma vez que está diretamente relacionada à contaminação de solos agrícolas e de recursos hídricos. Tradicionalmente, esses processos são descritos por **modelos cinéticos baseados em fundamentos físico-químicos** bem estabelecidos.

Com o avanço recente das **técnicas de Machine Learning**, cresce o interesse em aplicar modelos empíricos para descrever fenômenos ambientais, mesmo em cenários com conjuntos de dados reduzidos. No entanto, ainda existe uma lacuna quanto à real vantagem do uso de Machine Learning frente a modelos clássicos quando o fenômeno é bem compreendido e os dados são limitados.

Diante desse contexto, este projeto surge da motivação de investigar, de forma crítica e comparativa, se modelos de regressão baseados em Machine Learning são capazes de superar — ou ao menos se equiparar — a modelos cinéticos clássicos na descrição da degradação e lixiviação do fungicida mancozebe em solo, utilizando dados experimentais reais provenientes de um estudo ambiental controlado.

## Objetivo Geral do Projeto

Avaliar o desempenho de modelos de aprendizado de máquina na previsão da concentração de mancozebe no solo ao longo do tempo e da profundidade, comparando-os a um modelo cinético de primeira ordem, e analisar como o enriquecimento do dataset com propriedades físicas do solo influencia a capacidade preditiva dos modelos.

## Objetivos Específicos

- **Modelar a degradação do mancozebe no solo** utilizando um modelo cinético de primeira ordem para diferentes profundidades, estimando parâmetros como constante de degradação e meia-vida.

- **Desenvolver modelos de Machine Learning** (Regressão Linear, Regressão Polinomial e Random Forest) para prever a concentração de mancozebe com base em variáveis experimentais básicas (tempo e profundidade).

- **Comparar quantitativamente** o desempenho dos modelos cinéticos e de Machine Learning utilizando métricas de erro (RMSE e MAE).

- **Avaliar o impacto do enriquecimento do dataset** com variáveis físicas do solo (ex.: umidade, textura, teor de matéria orgânica) no desempenho dos modelos de Machine Learning.

- **Discutir o papel complementar** entre modelos mecanísticos e modelos baseados em dados na análise da dinâmica de pesticidas no solo.

## Descrição do Dataset
#### Estrutura do Dataset (inicial)

Os dados utilizados neste projeto foram obtidos diretamente a partir de experimentos realizados no contexto da pesquisa de mestrado, adequado para análises exploratórias, modelagem cinética e aplicação de modelos de Machine Learning.

- **dias**: tempo decorrido (em dias) após a aplicação do fungicida;
- **profundidade_cm**: profundidade de coleta da amostra de solo (em centímetros);
- **concentracao_mg_kg**: concentração de resíduos de mancozebe determinada por método espectrofotométrico;
- **censured**: variável indicadora de censura analítica, informando se a concentração medida encontra-se abaixo do limite de detecção (LD).

#### Variáveis físico-químicas do solo (dataset enriquecido)

- **Umidade do solo (%)**: quantidade de água retida nos poros do solo;
- **Composição granulométrica**: distribuição percentual das partículas minerais com base no tamanho (frações de areia, silte e argila);
- **Teor de carbono orgânico (%)**: é a medida da quantidade de carbono presente na matéria orgânica;
- **pH do solo**: é a medida da acidez ou alcalinidade (potencial hidrogeniônico) da solução do solo.

#### Características do Dataset

- **Baixo número de observações**, típico de experimentos ambientais controlados.
- Presença de **valores abaixo do limite de detecção**, exigindo tratamento estatístico adequado.
- Forte **componente físico-química**, favorecendo a comparação entre abordagens mecanísticas e modelos baseados em dados.

## Importação das bibliotecas


In [1]:
import pandas as pd
import numpy as np

## Preparação dos Dados


In [2]:
# Limite de detecção
LD = 0.20
LD_SUB = LD / 2  # 0.10 mg/kg

# Dados originais do experimento
data = [
    # dias, profundidade (cm), concentração, censurado?
    (2, 5, 3.80, False),
    (2, 15, 1.50, False),
    (2, 30, 0.99, False),

    (5, 5, 2.17, False),
    (5, 15, 0.77, False),
    (5, 30, 0.46, False),

    (8, 5, 0.83, False),
    (8, 15, 0.41, False),
    (8, 30, 0.55, False),

    (12, 5, 0.78, False),
    (12, 15, 0.39, False),
    (12, 30, LD_SUB, True),


    (15, 5, 0.53, False),
    (15, 15, 0.35, False),
    (15, 30, LD_SUB, True),

    (18, 5, 0.25, False),
    (18, 15, LD_SUB, True),
    (18, 30, LD_SUB, True),

    (35, 5, LD_SUB, True),
    (35, 15, LD_SUB, True),
    (35, 30, LD_SUB, True),
]

# Criando o DataFrame
df = pd.DataFrame(
    data,
    columns=["dias", "profundidade_cm", "concentracao_mg_kg", "censored"]
)


# Criar um dicionário com propriedades novas variáveis do solo

soil_properties = {
    5:  {"umidade": 23, "argila": 18.4, "areia": 16.8, "silte": 18.6, "carbono_org": 2.4, "pH": 5.6},
    15: {"umidade": 21.6, "argila": 18.4, "areia": 16.3, "silte": 18.4, "carbono_org": 2.0, "pH": 5.5},
    30: {"umidade": 20.8, "argila": 19.4, "areia": 23.1, "silte": 19.4, "carbono_org": 1.9, "pH": 5.6},
}

# Mapear essas propriedades para o dataset

df["umidade"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["umidade"])
df["argila"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["argila"])
df["areia"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["areia"])
df["silte"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["silte"])
df["carbono_org"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["carbono_org"])
df["pH"] = df["profundidade_cm"].map(lambda x: soil_properties[x]["pH"])


In [3]:
df

,dias,profundidade_cm,concentracao_mg_kg,censored,umidade,argila,areia,silte,carbono_org,pH
0,2,5,3.80,False,23.0,18.4,16.8,18.6,2.4,5.6
1,2,15,1.50,False,21.6,18.4,16.3,18.4,2.0,5.5
2,2,30,0.99,False,20.8,19.4,23.1,19.4,1.9,5.6
3,5,5,2.17,False,23.0,18.4,16.8,18.6,2.4,5.6
4,5,15,0.77,False,21.6,18.4,16.3,18.4,2.0,5.5
5,5,30,0.46,False,20.8,19.4,23.1,19.4,1.9,5.6
6,8,5,0.83,False,23.0,18.4,16.8,18.6,2.4,5.6
7,8,15,0.41,False,21.6,18.4,16.3,18.4,2.0,5.5
8,8,30,0.55,False,20.8,19.4,23.1,19.4,1.9,5.6
9,12,5,0.78,False,23.0,18.4,16.8,18.6,2.4,5.6


#### Justificativa
Em análises ambientais, valores reportados como abaixo do limite de detecção (LD) são comuns e refletem limitações inerentes ao método analítico, não a ausência do composto no meio. A exclusão desses dados ou sua substituição por zero pode introduzir vieses estatísticos e comprometer a interpretação físico-química do processo de degradação.

Neste projeto, os valores abaixo do LD foram tratados de forma a preservar a continuidade temporal do fenômeno e a coerência com o comportamento esperado da cinética de degradação, permitindo a aplicação de modelos matemáticos e de Machine Learning sem distorções artificiais.

Para valores abaixo do limite de detecção (LD = 0,20 mg·kg⁻¹) foram tratados como dados censurados. Para fins de modelagem, esses valores foram substituídos por LD/2, conforme prática consagrada na literatura de química ambiental, permitindo a preservação das observações no conjunto de dados. A influência dessa decisão nos resultados é discutida na seção de limitações.

In [4]:
df.describe()

,dias,profundidade_cm,concentracao_mg_kg,umidade,argila,areia,silte,carbono_org,pH
count,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000,21.000000
mean,13.571429,16.666667,0.689524,21.800000,18.733333,18.733333,18.800000,2.100000,5.566667
std,10.404669,10.527741,0.884734,0.931665,0.483046,3.170857,0.442719,0.221359,0.048305
min,2.000000,5.000000,0.100000,20.800000,18.400000,16.300000,18.400000,1.900000,5.500000
25%,5.000000,5.000000,0.100000,20.800000,18.400000,16.300000,18.400000,1.900000,5.500000
50%,12.000000,15.000000,0.410000,21.600000,18.400000,16.800000,18.600000,2.000000,5.600000
75%,18.000000,30.000000,0.780000,23.000000,19.400000,23.100000,19.400000,2.400000,5.600000
max,35.000000,30.000000,3.800000,23.000000,19.400000,23.100000,19.400000,2.400000,5.600000


In [5]:
import os

os.makedirs("../data/raw", exist_ok=True)

df.to_csv(
    "../data/raw/mancozebe_experimental.csv",
    index=False
)